<a href="https://colab.research.google.com/github/AkritiGhosh/DeeperSystemAssignment/blob/master/Task_2_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Assignment
Question link - https://gist.github.com/csaftoiu/9fccaf47fd8f96cd378afd8fdd0d63c1

By - Akriti Ghosh

Importing dataset using the link provided in the question link

In [ ]:
!wget https://www.dropbox.com/s/lbobq9xt3nchq5q/train.rotfaces.zip?dl=0 \
  -O /train.rotfaces.zip
!wget  https://www.dropbox.com/s/ustfubunhfe47mj/test.rotfaces.zip?dl=0 \
  -O /test.rotfaces.zip

--2020-08-13 06:03:52--  https://www.dropbox.com/s/lbobq9xt3nchq5q/train.rotfaces.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/lbobq9xt3nchq5q/train.rotfaces.zip [following]
--2020-08-13 06:03:53--  https://www.dropbox.com/s/raw/lbobq9xt3nchq5q/train.rotfaces.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7e3d7357b35be778adc799c7c7.dl.dropboxusercontent.com/cd/0/inline/A9UwqkD2vZx6CjEiKwB1CY32qHAigDC0nRmO-Ydqnf6MiYou-AVAXx5pLrRW0ly-I5wpkYphTEn4jrqD72d4wEa8DwLZlrijhRpl9JxCDmJ88A/file# [following]
--2020-08-13 06:03:54--  https://uc7e3d7357b35be778adc799c7c7.dl.dropboxusercontent.com/cd/0/inline/A9UwqkD2vZx6CjEiKwB1CY32qHAigDC0nRmO-Ydqnf6MiYou-AVAXx5pLrRW0ly-I5wpkYphTEn4jrqD72d4wEa8DwLZlrijhRpl9JxCDmJ88A/f

Unzipping the datasets and storing it in folder in colab file storage

In [ ]:
import zipfile
zip_train = zipfile.ZipFile('/train.rotfaces.zip','r')
zip_train.extractall('train.rotfaces')
zip_train.close()

zip_test = zipfile.ZipFile('/test.rotfaces.zip','r')
zip_test.extractall('test.rotfaces')
zip_test.close()

Importing the necessary libraries

In [ ]:
from google.colab import files # For downloading csv and h5 files
import csv # Editting the csv file
import pandas as pd # Mathematical manipulation of data
import glob # Accessing files in a folder
import numpy as np # Create and manipulate arrays
from PIL import Image # Display and manipulate images

from sklearn.preprocessing import LabelEncoder # To encode the labels from string to int 
from sklearn.model_selection import train_test_split # To split the training dataset for validation

from keras.utils import to_categorical # Categorization of labels in training data
from keras.preprocessing import image # Loading, Accessing & manipulating images 

# Models, Layers, Optimizers used (Tensorflow)
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
# Storing the path of training data images
train = 'train.rotfaces/train/'
# Storing the csv file as a Pandas DataFrame
train_df = pd.read_csv('/content/train.rotfaces/train.truth.csv')

# Data Preprocessing

1.  Encoding the labels from string value to integers
2.  Reading the images from training & testing dataset and storing them in a numpy array of names **X** & **x_test** respectively
3.  Storing the names of the images  in testing dataset in a list called **fn**
4.  Splitting the X and Y of training dataset into train & test data for validation in the ratio of 80-20



In [ ]:
encoder = LabelEncoder()
label = encoder.fit_transform(train_df.iloc[:, -1].values)
value = train_df.iloc[:, 1].values

In [ ]:
train_image = []
for i in (train_df.iloc[:,0]):
    img = image.load_img(train+ i,target_size=(64,64,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

In [ ]:
test_image = []
fn = []
for i in glob.glob('/content/test.rotfaces/test/*.jpg'):
  fn.append(i.split('/')[-1])
  img = image.load_img( i,target_size=(64,64,3))
  img = image.img_to_array(img)
  img = img/255
  test_image.append(img)

In [ ]:
X = np.array(train_image)
x_test = np.array(test_image)
Y = to_categorical(label)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,  test_size=0.2)

# Model Creation and Predictions
Two models are used for the classifiction of images.
1.  CIFAR10 Keras model - https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py

2. My Custom CNN model : This model uses lesser layers, and gives more accurate prediction in less time than the CIFAR_CNN model. In this, lesser number of Convolution layer & MaxPooling layers, and more Dense layer are used compared to the CIFAR10 model.

The Optimizers used are :
1. CIFAR10 - RMSProp
2. My CNN - Adam

The epoch & batch size of both the models are same, ie, 5 and 256 respectively


In [ ]:
batch_size = 256
epochs = 5

In [ ]:
cifar10 = Sequential()
cifar10.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X.shape[1:]))
cifar10.add(Activation('relu'))
cifar10.add(Conv2D(32, (3, 3)))
cifar10.add(Activation('relu'))
cifar10.add(MaxPooling2D(pool_size=(2, 2)))
cifar10.add(Dropout(0.25))

cifar10.add(Conv2D(64, (3, 3), padding='same'))
cifar10.add(Activation('relu'))
cifar10.add(Conv2D(64, (3, 3)))
cifar10.add(Activation('relu'))
cifar10.add(MaxPooling2D(pool_size=(2, 2)))
cifar10.add(Dropout(0.25))

cifar10.add(Flatten())
cifar10.add(Dense(512))
cifar10.add(Activation('relu'))
cifar10.add(Dropout(0.5))
cifar10.add(Dense(4))
cifar10.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the cifar10 using RMSprop
cifar10.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [ ]:
cifar10.fit(x=X_train,y= y_train, 
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)

scores = cifar10.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/5
153/153 [==============================] - 21s 138ms/step - loss: 0.9102 - accuracy: 0.6286 - val_loss: 0.5184 - val_accuracy: 0.8187
Epoch 2/5
153/153 [==============================] - 20s 130ms/step - loss: 0.4310 - accuracy: 0.8388 - val_loss: 0.3023 - val_accuracy: 0.8882
Epoch 3/5
153/153 [==============================] - 20s 131ms/step - loss: 0.2954 - accuracy: 0.8899 - val_loss: 0.2298 - val_accuracy: 0.9172
Epoch 4/5
153/153 [==============================] - 20s 130ms/step - loss: 0.2213 - accuracy: 0.9206 - val_loss: 0.1772 - val_accuracy: 0.9395
Epoch 5/5
306/306 [==============================] - 3s 9ms/step - loss: 0.1444 - accuracy: 0.9502
Test loss: 0.14444614946842194
Test accuracy: 0.9502044916152954


In [ ]:
my_cnn = tf.keras.models.Sequential([
                                    Conv2D(32, (3,3), activation='relu', input_shape = X.shape[1:], padding='same'),
                                    MaxPooling2D(2,2),
                                    Conv2D(32, (3,3), activation='relu'),
                                    MaxPooling2D(2,2),
                                    Conv2D(64, (3,3), activation='relu'),
                                    MaxPooling2D(2,2),
                                    Flatten(),
                                    Dense(128, activation='relu'),
                                    Dense(64, activation='relu'),
                                    Dense(4, activation='softmax')
])

In [ ]:
my_cnn.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
my_cnn.fit(x=X_train,y= y_train, 
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)

Epoch 1/5
153/153 [==============================] - 7s 46ms/step - loss: 0.6114 - accuracy: 0.7527 - val_loss: 0.3163 - val_accuracy: 0.8817
Epoch 2/5
153/153 [==============================] - 7s 43ms/step - loss: 0.2114 - accuracy: 0.9247 - val_loss: 0.1774 - val_accuracy: 0.9380
Epoch 3/5
153/153 [==============================] - 7s 43ms/step - loss: 0.1323 - accuracy: 0.9551 - val_loss: 0.1421 - val_accuracy: 0.9491
Epoch 4/5
153/153 [==============================] - 7s 43ms/step - loss: 0.0932 - accuracy: 0.9686 - val_loss: 0.1339 - val_accuracy: 0.9563
Epoch 5/5
153/153 [==============================] - 7s 43ms/step - loss: 0.0746 - accuracy: 0.9750 - val_loss: 0.0989 - val_accuracy: 0.9656


In [ ]:
my_cnn.summary()
# my_cnn_name = 'cnn_967_e5_b256_l10.h5'
# my_cnn.save(my_cnn_name)
# files.download(my_cnn_name)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)             

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# The testing accuracy of my_cnn model
scores = my_cnn.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

306/306 [==============================] - 1s 5ms/step - loss: 0.0989 - accuracy: 0.9656
Test loss: 0.0988747626543045
Test accuracy: 0.9656441807746887


# Using the model to correct the test dataset (the images in test.rotfaces)

1. Predicting values for x_test and saving it in y_pred
2. Predicting the rotation in faces using y_pred and saving the string value in test_label list
3. Rotating the images according to the predictions and storing the image arrays in a numpy array called correction
4. Mounting the Google Drive and storing the corrected images in the drive
5. Creating a DataFrame of test images file names (fn) and the predicted labels (test_labels) and storing it in a csv file called **test.preds.csv**

In [ ]:
y_pred = my_cnn.predict_classes(x_test)
cifar_pred = cifar10.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cifar_label = []
crrt_cifar_img = []
for i in range(len(cifar_pred)):
  img = Image.fromarray((x_test[i]*255).astype(np.uint8))
  # display(img)
  if cifar_pred[i]==0:
    # print('rotated_left')
    cifar_label.append('rotated_left')
    img = img.transpose(Image.ROTATE_270)
  
  elif cifar_pred[i]==1:
    # print('rotated_right')
    cifar_label.append('rotated_right')
    img = img.transpose(Image.ROTATE_90)
  
  elif cifar_pred[i]==2:
    # print('upright')
    cifar_label.append('upright')
    # img = img.transpose() No transpose required
  
  else:
    # print('rotated_left')
    cifar_label.append('upside_down')
    img = img.transpose(Image.ROTATE_180)
  
  fl_nm = str(fn[i].split('.')[0])+'.png'
  crrt_cifar_img.append(np.array(img))
  img = img.save('/content/drive/My Drive/taskNN/cifar_model/'+fl_nm)



In [ ]:
test_label = []
correct_img = []
for i in range(len(y_pred)):
  img = Image.fromarray((x_test[i]*255).astype(np.uint8))
  # display(img)
  if y_pred[i]==0:
    # print('rotated_left')
    test_label.append('rotated_left')
    img = img.transpose(Image.ROTATE_270)
  
  elif y_pred[i]==1:
    # print('rotated_right')
    test_label.append('rotated_right')
    img = img.transpose(Image.ROTATE_90)
  
  elif y_pred[i]==2:
    # print('upright')
    test_label.append('upright')
    # img = img.transpose() No transpose required
  
  else:
    # print('rotated_left')
    test_label.append('upside_down')
    img = img.transpose(Image.ROTATE_180)
  
  fl_nm = str(fn[i].split('.')[0])+'.png'
  correct_img.append(np.array(img))
  img = img.save('/content/drive/My Drive/taskNN/my_model/'+fl_nm)



In [ ]:
test_pred = pd.DataFrame(index = fn, data = test_label)
test_pred.to_csv('test.preds.csv')
files.download('test.preds.csv')

In [ ]:
correction = np.array(correct_img)
cifar_correction = np.array(crrt_cifar_img)

In [ ]:
test_pred = pd.DataFrame(index = fn, data = cifar_label)
test_pred.to_csv('ctest.preds.csv')
files.download('ctest.preds.csv')